<a href="https://colab.research.google.com/github/adrianpietrzak/ML_BootCamp/blob/main/supervised/01_basics/01_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Importujemy statystki

In [1]:
import numpy as np
import pandas as pd
import sklearn

In [2]:
sklearn.__version__

'1.2.2'

### Tworzymy dane i importujemy je jako DataFrame

In [3]:
data = {
    'size' : ['XL', 'L', 'M', 'L', 'M'],
    'color' : ['red', 'green', 'blue', 'green', 'red'],
    'gender' : ['female', 'male', 'male', 'female', 'female'],
    'price' : [199.0, 89.0, 99.0, 129.0, 79.0],
    'weight' : [500, 450, 300, 380, 410],
    'bought' : ['yes', 'no', 'yes', 'no', 'yes']
}

df_raw = pd.DataFrame(data=data)
df_raw

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500,yes
1,L,green,male,89.0,450,no
2,M,blue,male,99.0,300,yes
3,L,green,female,129.0,380,no
4,M,red,female,79.0,410,yes


### Tworzymy kopie danych

In [4]:
df = df_raw.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   size    5 non-null      object 
 1   color   5 non-null      object 
 2   gender  5 non-null      object 
 3   price   5 non-null      float64
 4   weight  5 non-null      int64  
 5   bought  5 non-null      object 
dtypes: float64(1), int64(1), object(4)
memory usage: 368.0+ bytes


### Zmieniamy typy danych z object na category oraz int64 na float64

In [5]:
for col in ['size', 'color', 'gender', 'bought']:
  df[col] = df[col].astype('category')

df['weight'] = df['weight'].astype('float')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   size    5 non-null      category
 1   color   5 non-null      category
 2   gender  5 non-null      category
 3   price   5 non-null      float64 
 4   weight  5 non-null      float64 
 5   bought  5 non-null      category
dtypes: category(4), float64(2)
memory usage: 740.0 bytes


###Analiza wstępna plus transpozycja - polecenie describe wyswietla bez include tylko dane liczbowe

In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
price,5.0,119.0,48.476799,79.0,89.0,99.0,129.0,199.0
weight,5.0,408.0,75.299402,300.0,380.0,410.0,450.0,500.0


### dzieki include wyswietla tez dane z typem category, .T to transpozycja

In [8]:
df.describe(include=['category']).T

,count,unique,top,freq
size,5,3,L,2
color,5,3,green,2
gender,5,2,female,3
bought,5,2,yes,3


In [9]:
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,yes
1,L,green,male,89.0,450.0,no
2,M,blue,male,99.0,300.0,yes
3,L,green,female,129.0,380.0,no
4,M,red,female,79.0,410.0,yes


### Przygotowanie danych do modelu, zamieniamy dane z category na liczbowe uzywajac biblioteki LabelEncoder

In [10]:
from sklearn.preprocessing import LabelEncoder # Stosujemy ta klase tylko do zmiennej bought (bo tu jest tylko yes/no), nie stosujemy jej do zmiennych kategorycznych

le = LabelEncoder() # Tworzymy instancje klasy LabelEncoder
le.fit(df['bought']) # metoda fit dopasowyuje do naszej zmiennej
le.transform(df['bought']) # transformacja danych do modelu

array([1, 0, 1, 0, 1])

In [11]:
le.fit_transform(df['bought']) # robi fit i transform za jednym zamachem

array([1, 0, 1, 0, 1])

In [13]:
le.classes_ # ten atrybut pokazuje jak to mapowanie wygląda, 0 = NO, 1 = YES

array(['no', 'yes'], dtype=object)

In [14]:
df['bought'] = le.fit_transform(df['bought']) # tutaj na stale przypisujemy mapowanie do datafrejmu DF
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,1
1,L,green,male,89.0,450.0,0
2,M,blue,male,99.0,300.0,1
3,L,green,female,129.0,380.0,0
4,M,red,female,79.0,410.0,1


In [25]:
le.inverse_transform(df['bought']) #gdybysmy chcieli wrocic do poprzedniej postaci bo cos by bylo nie tak

array([1, 0, 1, 0, 1])

In [16]:
df['bought'] = le.fit_transform(df['bought']) # tutaj na stale przypisujemy mapowanie do datafrejmu DF - musze zrobic jeszcze raz ze wzgledu na poprzedni inverse
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,1
1,L,green,male,89.0,450.0,0
2,M,blue,male,99.0,300.0,1
3,L,green,female,129.0,380.0,0
4,M,red,female,79.0,410.0,1


### OneHotEncoder - mniej polecana niz Get Dummies

In [18]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False) # tworzymy instancje klasy Encoder, parametr sparse = false,
                                      #bo klasa zwraca nam macierz rzadką i nie jestesmy w stanie podejrzec jak ona wyglada, macierz rzadka - bo jak mamy duzo 0 i 1 to nie ma sensu w pamieci trzymac 0, sa trzymane jedynie 1
encoder.fit(df[['size']]) # dopasowujemy zmienne z df[size] do encodera

OneHotEncoder(sparse_output=False)

In [19]:
encoder.transform(df[['size']]) #robimy transform (jak wczesniej)

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [21]:
encoder.categories_ # pokazuje kategorie - tam gdzie w pierwszej kolumnie mamy L jest 1, w drugiej M jest 1, w trzeciej XL jest 1

[array(['L', 'M', 'XL'], dtype=object)]

In [22]:
encoder = OneHotEncoder(drop='first', sparse_output=False) #przy takim kodowaniu moze byc blad kodowania 0 - 1, bo nie powinno byc liniowej zaleznosci miedzy zmiennymi. Przekazujemy parametr drop='first' i usuwa on pierwsza kolumne
encoder.fit(df[['size']]) # dopasowujemy zmienne z df[size] do encodera
encoder.transform(df[['size']]) #robimy transform (jak wczesniej)

array([[0., 1.],
       [0., 0.],
       [1., 0.],
       [0., 0.],
       [1., 0.]])

In [26]:
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,1
1,L,green,male,89.0,450.0,0
2,M,blue,male,99.0,300.0,1
3,L,green,female,129.0,380.0,0
4,M,red,female,79.0,410.0,1


### Get Dummies - tez preksztalca, jest lepsza

In [27]:
pd.get_dummies(data=df) #odpala metode dummies

,price,weight,bought,size_L,size_M,size_XL,color_blue,color_green,color_red,gender_female,gender_male
0,199.0,500.0,1,0,0,1,0,0,1,1,0
1,89.0,450.0,0,1,0,0,0,1,0,0,1
2,99.0,300.0,1,0,1,0,1,0,0,0,1
3,129.0,380.0,0,1,0,0,0,1,0,1,0
4,79.0,410.0,1,0,1,0,0,0,1,1,0


In [28]:
pd.get_dummies(data=df, drop_first=True) #usuwa po pierwszej kolumnie, jak size_L, color_blue i gender_female, wiec informacje sie nie powielaja

,price,weight,bought,size_M,size_XL,color_green,color_red,gender_male
0,199.0,500.0,1,0,1,0,1,0
1,89.0,450.0,0,0,0,1,0,1
2,99.0,300.0,1,1,0,0,0,1
3,129.0,380.0,0,0,0,1,0,0
4,79.0,410.0,1,1,0,0,1,0


In [30]:
pd.get_dummies(data=df, drop_first=True, prefix='new') #dodaje prefix new do nowo utworzonych kolumn

,price,weight,bought,new_M,new_XL,new_green,new_red,new_male
0,199.0,500.0,1,0,1,0,1,0
1,89.0,450.0,0,0,0,1,0,1
2,99.0,300.0,1,1,0,0,0,1
3,129.0,380.0,0,0,0,1,0,0
4,79.0,410.0,1,1,0,0,1,0


In [31]:
pd.get_dummies(data=df, drop_first=True, prefix_sep='-') #ustawia separator

,price,weight,bought,size-M,size-XL,color-green,color-red,gender-male
0,199.0,500.0,1,0,1,0,1,0
1,89.0,450.0,0,0,0,1,0,1
2,99.0,300.0,1,1,0,0,0,1
3,129.0,380.0,0,0,0,1,0,0
4,79.0,410.0,1,1,0,0,1,0


In [32]:
pd.get_dummies(data=df, drop_first=True, columns=['size'])  #ustawia kodowanie dla tylko jednej kolumny

,color,gender,price,weight,bought,size_M,size_XL
0,red,female,199.0,500.0,1,0,1
1,green,male,89.0,450.0,0,0,0
2,blue,male,99.0,300.0,1,1,0
3,green,female,129.0,380.0,0,0,0
4,red,female,79.0,410.0,1,1,0


###Standaryzacja danych numerycznych - Standard Scaler
skala wartosci nie ma znaczenie, ma znaczenie ich rozrzut czyli wariacja, musi byc maly rozrzut

std() - pandas nieobciazony albo
std() - numpy obciazany

In [38]:
print(f"{df['price']}\n") #Estymator nie obciazony dla biblioteki Pandas, tak wyglada srednia i odchylenie
print(f"Srednia: {df['price'].mean()}")
print(f"Odchylenie standardowe: {df['price'].std():.4f}")

0    199.0
1     89.0
2     99.0
3    129.0
4     79.0
Name: price, dtype: float64

Srednia: 119.0
Odchylenie standardowe: 48.4768


In [35]:
(df['price'] - df['price'].mean()) / df['price'].std() # tak liczmy manualnie odchylenie standardowe

0    1.650274
1   -0.618853
2   -0.412568
3    0.206284
4   -0.825137
Name: price, dtype: float64

In [36]:
def standarize(x):
  return (x - x.mean()) / x.std()

standarize(df['price']) # budujemy wlasna funkcje liczaca odchylenie standardowe zeby nie musiec liczyc manualnie

0    1.650274
1   -0.618853
2   -0.412568
3    0.206284
4   -0.825137
Name: price, dtype: float64

In [37]:
from sklearn.preprocessing import scale #zawiera gotowa funkcje do liczenia odchylenia standardowego, dane sa inne niz wczesniej a jest to kwestia estymatora obciazonego

scale(df['price'])

array([ 1.84506242, -0.69189841, -0.4612656 ,  0.2306328 , -0.92253121])

In [40]:
from sklearn.preprocessing import StandardScaler #ta klasa polecana bardziej do standaryzacji niż scale

scaler = StandardScaler()
scaler.fit(df[['price']]) # DOPASOWUJEMY
scaler.transform(df[['price']]) # TRANSFORMACJA

array([[ 1.84506242],
       [-0.69189841],
       [-0.4612656 ],
       [ 0.2306328 ],
       [-0.92253121]])

In [43]:
scaler = StandardScaler()
df[['price','weight']] = scaler.fit_transform(df[['price','weight']]) #robimy fit and transform danych dla obu kolumn na raz
df

,size,color,gender,price,weight,bought
0,XL,red,female,1.845062,1.366002,1
1,L,green,male,-0.691898,0.623610,0
2,M,blue,male,-0.461266,-1.603567,1
3,L,green,female,0.230633,-0.415740,0
4,M,red,female,-0.922531,0.029696,1


#POWTORKA - przygotowanie danych do Modelu

In [44]:
df = df_raw.copy()
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500,yes
1,L,green,male,89.0,450,no
2,M,blue,male,99.0,300,yes
3,L,green,female,129.0,380,no
4,M,red,female,79.0,410,yes


In [45]:
le = LabelEncoder() #pozwala zakodowac zmienna docelowa, bo mamy wartosci True False = 0, 1

df['bought'] = le.fit_transform(df['bought'])

scaler = StandardScaler() #standaryzujemy zmienne numeryczne
df[['price','weight']] = scaler.fit_transform(df[['price','weight']])

df = pd.get_dummies(data=df, drop_first=True) #zmienne kategoryczne zmieniamy, z drop firstem aby usunac niepotrzebne kolumny
df

,price,weight,bought,size_M,size_XL,color_green,color_red,gender_male
0,1.845062,1.366002,1,0,1,0,1,0
1,-0.691898,0.623610,0,0,0,1,0,1
2,-0.461266,-1.603567,1,1,0,0,0,1
3,0.230633,-0.415740,0,0,0,1,0,0
4,-0.922531,0.029696,1,1,0,0,1,0
